# Imports the libraries and functions that will be necessary to create mind maps from PDFs.

In [ ]:
import time
import google.generativeai as genai
import os
import time
import datetime
from IPython.display import Image, display
from dotenv import load_dotenv

### DEFINES FUNCTION THAT CHECKS IF THERE ARE FILES UPLOADED TO THE GENAI CLOUD
def check_files_in_genai_cloud():
    global filesExistInCloud 
    cloud_file_list = []
    for file in genai.list_files():
        cloud_file_list.append(file)

    if not cloud_file_list:
        print('There are no files in the Genai cloud.')
        filesExistInCloud = False
    else:
        print('There are files in the Genai cloud.')
        print(genai.list_files())
        filesExistInCloud = True

### DEFINES FUNCTION THAT WILL REQUEST THE RESPONSE FROM GEMINI AND FORMAT IT IN PUML MINDMAP
def run_API():
    global str_puml
    global lines
    response = model.generate_content([file, prompt_text])
    # Tries to ensure that Gemini's response is indeed in .puml format:
    lines = response.text.split('\n')
    
    # Check if the first line doesn't start with '*' or '@startmindmap'
    if not lines[0].startswith('*') and not lines[0] == '@startmindmap':
        # Remove the first line and add '@startmindmap' as the first line
        lines = ['@startmindmap'] + lines[1:]
    
    # Check if the first line starts with '*' 
    if lines[0].startswith('*'):
        # Add a line at the beginning with '@startmindmap'
        lines = ['@startmindmap'] + lines
    
    # Check if the last line doesn't start with '*' or '@endmindmap'
    if not lines[-1].startswith('*') and not lines[-1] == '@endmindmap':
        # Remove the last line and add '@endmindmap' as the last line
        lines = lines[0:-1] + ['@endmindmap']
    
    # Check if the last line starts with '*' 
    if lines[-1].startswith('*'):
        # Add a line at the end with '@endmindmap'
        lines = lines + ['@endmindmap']
    
    lines[1:-1] = [line for line in lines[1:-1] if line.startswith('*')]
            
    # Join the lines back into a string
    str_puml = '\n'.join(lines)
    # Check if it met the criteria
    check_criteria()

### DEFINES FUNCTION THAT WILL CHECK IF THE PUML MINDMAP GENERATED BY GEMINI REACHED THE 4TH LEVEL, WHICH IS A LINE STARTING WITH "**** "
def check_criteria():
    global var_end
    # Initialization of the criteria_error_counter variable
    if 'criteria_error_counter' not in globals():
        global criteria_error_counter
        criteria_error_counter = 0
    var_end = False
    for line in lines:
        if line.startswith("**** "):
            var_end = True
    if var_end==True:
        generate_mind_map()
        criteria_error_counter = 0
    else:
        print("The .puml generated by Gemini does not meet the criteria. Suspension initiated for 10 seconds...")
        time.sleep(10)
        print("10 seconds have passed. Execution resumed. Let's try to generate a mind map with Gemini again.")
        criteria_error_counter += 1
        if criteria_error_counter < 5:
            run_API()
        else:
            criteria_error_counter = 0
            print(f"Unable to generate mind map for file {file}. Gemini generated a .puml that did not meet the criterion of having at least 4 levels 5 times.")

### DEFINES FUNCTION THAT WILL GENERATE THE MIND MAP
def generate_mind_map():
    # Generates the mind map in .png format
    print("Attempt to generate the map started.")

    # Generate the date and time in the desired format
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # .puml file name with date and time
    path_puml = f"outputs\\{timestamp}_mapa_mental_{os.path.splitext(file.display_name)[0]}.puml"
    
    # Generate the output PNG file name
    path_png = path_puml[:-4]+"png"
    
    # Create the PlantUML file with the mind map
    with open(path_puml, "w") as f:
        f.write(str_puml)
    
    # Step 2: Run PlantUML locally to generate the image
    plantuml_jar_path = "plantuml_jar\\plantuml-1.2024.7.jar"
    command = f"java -Dfile.encoding=UTF-8 -jar \"{plantuml_jar_path}\" \"{path_puml}\""
    
    # Execute the command and check the return
    exit_code = os.system(command)
    
    if exit_code == 0:
        print("Image generated successfully!")
    else:
        print("There was an error generating the image.")
    
    # Step 3: Display the image in Jupyter if it was generated correctly
    if os.path.exists(path_png):
        display(Image(path_png))
        error_counter=0
    else:
        print("Image not found. Check if PlantUML correctly converted the .puml file to .png.")
    # Suspend execution for 10 seconds
    print("10-second suspension initiated...")
    time.sleep(10)

# Initializes the Genai model and uploads PDFs to the Genai cloud
Run this code to send the PDFs from your machine, located in the "pdf" folder of this project, to Google's Genai cloud.

In [ ]:
load_dotenv("keys.env")  # loads variables from keys.env file
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

#### CONFIGURES THE LARGE LANGUAGE MODEL (LLM) GEMINI ####
genai.configure(api_key=GOOGLE_API_KEY) # Configures genai with GOOGLE_API_KEY
model = genai.GenerativeModel(model_name="gemini-1.5-flash") # Defines the Gemini model.

check_files_in_genai_cloud() # Checks if there are files in the Genai cloud

#### REMOVES ALL FILES THAT WERE UPLOADED TO THE GENAI CLOUD (google.generativeai) ####
if filesExistInCloud:
    for file in genai.list_files(): # Starts the loop in the list of files in the genai cloud
        genai.delete_file(file.name) # Delete file from genai cloud
        print(f"The file {file.display_name} was removed from the cloud.")

check_files_in_genai_cloud() # Checks if there are files in the Genai cloud

#### UPLOADS FILES TO THE GENAI CLOUD (google.generativeai) ####
inputs_path = "inputs" # Defines the directory where PDF files are located
files = [os.path.join(inputs_path, pdf) for pdf in os.listdir(inputs_path)] # Lists all files in inputs_path with full path
files.sort(key=os.path.getctime) # Sorts files by creation date
for path in files: # Starts the loop in the list of full file paths
    pdf_name = os.path.basename(path) # gets the file name
    if pdf_name.endswith('.pdf'):  # Checks if the file is a PDF      
        file = genai.upload_file(path=path, display_name=pdf_name)  # Uploads the file   
        print(f"Uploaded file '{file.display_name}' as: {file.uri}")
        time.sleep(1) # waits 1 second before iterating again, to facilitate viewing the difference in file creation dates in the cloud
uploaded_files = list(genai.list_files())  # creates the list of uploads using the genai.list_files() generator
sorted_files = sorted(uploaded_files, key=lambda file: file.create_time) # Sorts the list of uploads by the 'create_time' attribute
for file in sorted_files: # Displays the sorted list of uploads
    print(f"{file.display_name} - {file.create_time}")

# Defines the prompt that will be sent with each PDF to Gemini and executes all necessary functions to generate mind maps in .png format

In [ ]:
#### SENDS PROMPTS TO GEMINI AND GENERATES MIND MAPS ####
for file in sorted_files:
    global error_counter
    error_counter = 0
    error_list = []
    try:
        # Code that may generate an error
        prompt_text = '''
        Please create a mind map of this PDF in .puml format with topics and subtopics to be used in plantuml, without indentations.
        Ensure that the .puml file contains no more than 90 lines. You are encouraged to fill 60-80 lines.
        Maintain the language of the PDF.
        The first level should start with '* '. The second level should start with '** ', and so on.
        Do not use '-' at the beginning of each topic.
        The map should have at least 4 levels.
        At the first level, 1 to 10 words are allowed. Try to use this maximum word limit I am giving you to explore concepts.
        At the second level, 1 to 10 words are allowed. Try to use this maximum word limit I am giving you to explore concepts.
        At the third level, 1 to 20 words are allowed. You are encouraged to form phrases in this level.
        At the fourth level, 1 to 20 words are allowed. You are encouraged to form phrases in this level.
        Do not include final considerations, complementary materials, bibliographic references, or other topics that do not explicitly explain concepts.
        '''
        var_end = False
        while var_end == False:
            run_API()

    except Exception as e:
        error_counter += 1
        # Error handling (can be a log or print)
        print(f"An error occurred with item {file.display_name}: {e}")
        if error_counter < 5:
            run_API()
        else:
            error_list.append(file.display_name)
        # Move to the next iteration
        continue
print("Script finished! All files in the 'pdf' folder have been scanned to generate mind maps.")